In [1]:
# run this cell to mount your google drive and follow the instructions 
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# show your CPU and GPU info 
!nvidia-smi
!cat /proc/cpuinfo
!rm -rf sample_data

# install LightGBM model 
!git clone --recursive https://github.com/Microsoft/LightGBM.git
!cd LightGBM && rm -rf build && mkdir build && cd build && cmake -DUSE_GPU=1 ../../LightGBM && make -j4 && cd ../python-package && python3 setup.py install --precompile --gpu;

Mon May  3 11:38:47 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
# run this cell directly if the file is stored in your drive
# otherwise, rewrite the directory and/or file name
!cp /content/drive/MyDrive/0403.zip 0403.zip
!unzip /content/drive/MyDrive/0403.zip

Archive:  /content/drive/MyDrive/0403.zip
  inflating: 0403/5cd56b6ee2acfd2d33b5a247_test.csv  
  inflating: 0403/5cd56b7de2acfd2d33b5c14b_train.csv  
  inflating: 0403/5cd56b6ae2acfd2d33b59c90_test.csv  
  inflating: 0403/5cd56b7de2acfd2d33b5c14b_test.csv  
  inflating: 0403/5cd56b6ee2acfd2d33b5a247_train.csv  
  inflating: 0403/5cd56b6ae2acfd2d33b59c90_train.csv  
  inflating: 0403/5cd56b6fe2acfd2d33b5a386_test.csv  
  inflating: 0403/5cd56b6ae2acfd2d33b59ccc_test.csv  
  inflating: 0403/5cd56b5ae2acfd2d33b58546_train.csv  
  inflating: 0403/5cd56b5ae2acfd2d33b58549_test.csv  
  inflating: 0403/5cd56b5ae2acfd2d33b58549_train.csv  
  inflating: 0403/5cd56b6fe2acfd2d33b5a386_train.csv  
  inflating: 0403/5cd56b6ae2acfd2d33b59ccc_train.csv  
  inflating: 0403/5cd56b5ae2acfd2d33b58546_test.csv  
  inflating: 0403/5da1383b4db8ce0c98bc11ab_test.csv  
  inflating: 0403/5d27099f03f801723c32511d_test.csv  
  inflating: 0403/5d27096c03f801723c31e5e0_test.csv  
  inflating: 0403/5d27097f03f8017

In [4]:
import numpy as np
import pandas as pd
import scipy.stats as stats
from pathlib import Path
import glob

from sklearn.model_selection import KFold
import lightgbm as lgb

import psutil
import random
import os
import time
import sys
import math
from contextlib import contextmanager

# 10-fold cross validation
N_SPLITS = 10 

# set random seed 
SEED = 42

@contextmanager
# get training time for each fold 
def timer(name: str):
    t0 = time.time()
    p = psutil.Process(os.getpid())
    m0 = p.memory_info()[0] / 2. ** 30
    try:
        yield
    finally:
        m1 = p.memory_info()[0] / 2. ** 30
        delta = m1 - m0
        sign = '+' if delta >= 0 else '-'
        delta = math.fabs(delta)
        print(f"[{m1:.1f}GB({sign}{delta:.1f}GB): {time.time() - t0:.3f}sec] {name}", file=sys.stderr)


def set_seed(seed=42):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    
# MSE - evaluation metric
def comp_metric(xhat, yhat, fhat, x, y, f):
    intermediate = np.sqrt(np.power(xhat-x, 2) + np.power(yhat-y, 2)) + 15 * np.abs(fhat-f)
    return intermediate.sum()/xhat.shape[0]





set_seed(SEED)

# read training and test files 
feature_dir = './*/'
train_files = sorted(glob.glob(os.path.join(feature_dir, '*_train.csv')))
test_files = sorted(glob.glob(os.path.join(feature_dir, '*_test.csv')))

# parameters for the three LightGBM models
# x and y model 
lgb_params = {'objective': 'root_mean_squared_error',
              'boosting_type': 'gbdt',
              'n_estimators': 50000,
              'learning_rate': 0.05,
              'num_leaves': 90,
              'device': 'gpu',
              'colsample_bytree': 0.4,
              'subsample': 0.6,
              'subsample_freq': 2,
              'bagging_seed': SEED,
              'reg_alpha': 2,
              'reg_lambda': 2,
              'random_state': SEED,
              'n_jobs': -1
              }

# f model 
lgb_f_params = {'objective': 'multiclass',
                'boosting_type': 'gbdt',
                'n_estimators': 50000,
                'learning_rate': 0.05,
                'num_leaves': 90,
                'device': 'gpu',
                'colsample_bytree': 0.4,
                'subsample': 0.6,
                'subsample_freq': 2,
                'bagging_seed': SEED,
                'reg_alpha': 10,
                'reg_lambda': 2,
                'random_state': SEED,
                'n_jobs': -1
                }

# Calculate all data

In [5]:

for n_files, file in enumerate(train_files):
    data = pd.read_csv(file, index_col=0)
    test_data = pd.read_csv(test_files[n_files], index_col=0)

    # initialize three arrays for x, y and f to store predictions on validation set
    oof_x, oof_y, oof_f = np.zeros(data.shape[0]), np.zeros(data.shape[0]), np.zeros(data.shape[0])
    
    # temporary variable to store x and y prediction on test set 
    preds_x, preds_y = 0, 0
    
    # an array to store f predictions on test set for each fold
    preds_f_arr = np.zeros((test_data.shape[0], N_SPLITS))
    
    print("Starting to train: ", os.path.basename(file))
    kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
    for fold, (trn_idx, val_idx) in enumerate(kf.split(data.iloc[:, :-4])):
        X_train = data.iloc[trn_idx, :-5] # all wifi signal info for training 
        y_trainx = data.iloc[trn_idx, -5] # true x coordinate for training 
        y_trainy = data.iloc[trn_idx, -4] # true y coordinate for training 
        y_trainf = data.iloc[trn_idx, -3] # true f for training 

        X_valid = data.iloc[val_idx, :-5] # all wifi signal info for validation set
        y_validx = data.iloc[val_idx, -5] # true x coordinate for validation 
        y_validy = data.iloc[val_idx, -4] # true y coordinate for validation 
        y_validf = data.iloc[val_idx, -3] # true f for validation 

        modelx = lgb.LGBMRegressor(**lgb_params) # model selected, which is LightGBM
        with timer("fit X"):
            modelx.fit(X_train, y_trainx,
                       eval_set=[(X_valid, y_validx)],
                       eval_metric='rmse',
                       verbose=False,
                       early_stopping_rounds=20
                       ) # Model 1, used to predit X location only

        modely = lgb.LGBMRegressor(**lgb_params)
        with timer("fit Y"):
            modely.fit(X_train, y_trainy,
                       eval_set=[(X_valid, y_validy)],
                       eval_metric='rmse',
                       verbose=False,
                       early_stopping_rounds=20
                       ) # Model 2, used to predit Y location only

        modelf = lgb.LGBMClassifier(**lgb_f_params)
        with timer("fit F"):
            modelf.fit(X_train, y_trainf,
                       eval_set=[(X_valid, y_validf)],
                       eval_metric='multi_logloss',
                       verbose=False,
                       early_stopping_rounds=20
                       ) # Model 3, used to predit Floor location only

        oof_x[val_idx] = modelx.predict(X_valid)
        oof_y[val_idx] = modely.predict(X_valid)
        oof_f[val_idx] = modelf.predict(X_valid).astype(int)

        # for x and y, average the result of each folds, divide every result by n_splits, and add them together
        preds_x += modelx.predict(test_data.iloc[:, :-5]) / N_SPLITS 
        preds_y += modely.predict(test_data.iloc[:, :-5]) / N_SPLITS
        
        # store floor predictions 
        preds_f_arr[:, fold] = modelf.predict(test_data.iloc[:, :-5]).astype(int)
           
        # compute MSE for validation set at current fold 
        score = comp_metric(oof_x[val_idx], oof_y[val_idx], oof_f[val_idx],
                            y_validx.to_numpy(), y_validy.to_numpy(), y_validf.to_numpy())
      
        print(f"fold {fold}: mean position error {score}")


    # evaluate MSE of each file
    print("*+"*40)
    print(f"file #{n_files}, shape={data.shape}, name={os.path.basename(file)}")
    score = comp_metric(oof_x, oof_y, oof_f,
                        data.iloc[:, -5].to_numpy(), data.iloc[:, -4].to_numpy(), data.iloc[:, -3].to_numpy())
    
    print(f"mean position error {score}")
    print("*+"*40)

    # the floor is dealt differently, vote to choose the most appeared floor number in all folds
    preds_f_mode = stats.mode(preds_f_arr, axis=1)
    preds_f = preds_f_mode[0].astype(int).reshape(-1)
    
    # stack all the predictions together, and transpose to vertical
    test_preds = pd.DataFrame(np.stack((preds_f, preds_x, preds_y))).T
    
    column_num = len(test_data.columns)
    test_data = test_data.reset_index()

    # insert all the predictions to test file
    test_data.insert(column_num, 'pred_x', test_preds[1], True)
    test_data.insert(column_num+1, 'pred_y', test_preds[2], True)
    test_data.insert(column_num+2, 'pred_f', test_preds[0], True) 

    building_name = file[7:-10] + '.csv'

    test_data.to_csv(building_name)

print("*+"*40)


Starting to train:  5cd56b5ae2acfd2d33b58544_train.csv


[0.3GB(+0.1GB): 8.217sec] fit X
[0.3GB(+0.0GB): 2.489sec] fit Y
[0.3GB(+0.0GB): 7.443sec] fit F


fold 0: mean position error 2.712883407509078


[0.3GB(+0.0GB): 2.735sec] fit X
[0.3GB(+0.0GB): 4.058sec] fit Y
[0.3GB(+0.0GB): 3.952sec] fit F


fold 1: mean position error 2.2441588290501167


...

fold 7: mean position error 3.5078898954117026


[2.8GB(+0.0GB): 42.897sec] fit X
[2.8GB(+0.0GB): 46.758sec] fit Y
[2.8GB(+0.0GB): 170.106sec] fit F


fold 8: mean position error 3.5071463867731154


[2.8GB(+0.0GB): 41.125sec] fit X
[2.8GB(+0.0GB): 45.596sec] fit Y
[2.8GB(+0.0GB): 163.977sec] fit F


fold 9: mean position error 3.575712733918063
*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+
file #23, shape=(11361, 1365), name=5da958dd46f8266d0737457b_train.csv
mean position error 3.5230187795118484
*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+
*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+


In [7]:
pred_files = glob.glob('/content' + '/*.csv')

In [8]:
# extract all true and predicted x, y and f
all_pred = pd.DataFrame()

for file_name in pred_files:
  file = pd.read_csv(file_name)

  all_pred = pd.concat([all_pred, file.iloc[:, -8:]])


In [9]:
# compute overall MSE
comp_metric(all_pred['pred_x'], all_pred['pred_y'], all_pred['pred_f'], all_pred['x'], all_pred['y'], all_pred['f'])

4.358951429786307